# Smart pointers

1. Pointers and ownership
    1. Raw pointer
    2. Reference
    3. Ownership
    4. Smart pointers
        1. `unique_ptr`
        2. `shared_ptr`
2. Revisit shared pointer
    1. Make Data exclusively managed by `shared_ptr`
    2. Get `shared_ptr` from `this`
    3. Cyclic reference and `weak_ptr`

# Raw pointer

The raw pointer allows us to directly manipulate the memory.  See the example file `01_pointer/01_raw_pointer.cpp`:

```cpp
struct PlainData
{
    int buffer[1024*8];
}; /* end struct PlainData */

int main(int argc, char ** argv)
{
    PlainData * ptr = nullptr;

    ptr = static_cast<PlainData *>(malloc(sizeof(PlainData)));
    std::cout << "PlainData pointer after malloc: " << ptr << std::endl;

    free(ptr);
    std::cout << "PlainData pointer after free: " << ptr << std::endl;

    ptr = new PlainData();
    std::cout << "PlainData pointer after new: " << ptr << std::endl;

    delete ptr;
    std::cout << "PlainData pointer after delete: " << ptr << std::endl;
}
```

In [ ]:
!make -C 01_pointer 01_raw_pointer ; 01_pointer/01_raw_pointer

# Reference

When we see a reference, we know that we should not deallocate / destruct the object.  See the example `01_pointer/02_reference.cpp`:

```cpp
struct PlainData
{
    int buffer[1024*8];
}; /* end struct PlainData */

// The factory function for PlainData.
PlainData * make_data()
{
    PlainData * ptr = new PlainData();
    // In a real factory function, we will do something with the allocated data
    // object before returning it.
    return ptr;
}

void manipulate_with_reference(PlainData & data)
{
    std::cout << "Manipulate with reference: " << &data << std::endl;

    for (size_t it=0; it < 1024*8; ++it)
    {
        data.buffer[it] = it;
    }
    // In a real consumer function we will do much more meaningful operations.

    // However, we cannot destruct an object passed in with a reference.
}

int main(int argc, char ** argv)
{
    PlainData * ptr = nullptr;

    ptr = make_data();
    std::cout << "PlainData pointer after factory: " << ptr << std::endl;

    manipulate_with_reference(*ptr);

    // Destruct the object where we have the pointer.
    delete ptr;
    std::cout << "PlainData pointer after delete: " << ptr << std::endl;
}
```

In [ ]:
!make -C 01_pointer 02_reference ; 01_pointer/02_reference

# Ownership

In a compliated system, memory is not free immediately after allocation.  Consider example `01_pointer/03_ownership.cpp`, where there are two worker functions with different memory management behaviors.

## Data class

Our data object is large, and we don't want the expensive overhead from frequent allocation and deallocation.

```cpp
class Data
{

public:

    constexpr const static size_t NELEM = 1024*8;

    using iterator = int *;
    using const_iterator = const int *;

    Data()
    {
        std::fill(begin(), end(), 0);
        std::cout << "Data @" << this << " is constructed" << std::endl;
    }

    ~Data()
    {
        std::cout << "Data @" << this << " is destructed" << std::endl;
    }

    const_iterator cbegin() const { return m_buffer; }
    const_iterator cend() const { return m_buffer+NELEM; }
    iterator begin() { return m_buffer; }
    iterator end() { return m_buffer+NELEM; }

    size_t size() const { return NELEM; }
    int   operator[](size_t it) const { return m_buffer[it]; }
    int & operator[](size_t it)       { return m_buffer[it]; }

    bool is_manipulated() const
    {
        for (size_t it=0; it < size(); ++it)
        {
            if ((*this)[it] != it) { return false; }
        }
        return true;
    }

private:

    // A lot of data that we don't want to reconstruct.
    int m_buffer[NELEM];

}; /* end class Data */

void manipulate_with_reference(Data & data)
{
    std::cout << "Manipulate with reference: " << &data << std::endl;

    for (size_t it=0; it < data.size(); ++it)
    {
        data[it] = it;
    }
    // In a real consumer function we will do much more meaningful operations.

    // However, we cannot destruct an object passed in with a reference.
}
```

## Separate memory management

The memory allocation and deallocation is not consistent in `worker1` and `worker2`.  This kind of problems are commonplace. 

```cpp
Data * worker1()
{
    // Create a new Data object.
    Data * data = new Data();

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

/*
 * Code in this function is intentionally made to be lack of discipline to
 * demonstrate how ownership is messed up.
 */
void worker2(Data * data)
{
    // The prerequisite for the caller to write correct code is to read the
    // code and understand when the object is alive.
    if (data->is_manipulated())
    {
        delete data;
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    Data * data = worker1();
    std::cout << "Data pointer after worker 1: " << data << std::endl;
    worker2(data);
    std::cout << "Data pointer after worker 2: " << data << std::endl;

    // You have to read the code of worker2 to know that data could be
    // destructed.  In addition, the Data class doesn't provide a
    // programmatical way to detect whether or not the object is alive.  The
    // design of Data, worker1, and worker2 makes it impossible to write
    // memory-safe code.
#ifdef CRASHME // The fenced code causes double free.
    delete data;
    std::cout << "Data pointer after delete: " << data << std::endl;
#endif
}
```

In [ ]:
!make -C 01_pointer 03_ownership ; 01_pointer/03_ownership

In [ ]:
# See how it crashes with double free.
!make -C 01_pointer clean
!env FLAGS=-DCRASHME make -C 01_pointer 03_ownership ; 01_pointer/03_ownership

## What is ownership

The above example shows the problem of lack of ownership.  "Ownership" isn't officially a language construct in C++, but is a common concept in many programming language for dynamic memory management.

To put it simply, when the object is "owned" by a construct or piece of code, it is assumed that it is safe for the piece of code to use that object.  The ownership assures the life of the object, and the object is not destructed when it is owned by someone.  It also means that the owner is reponsible for making sure the object gets destructed when it should be.

As we observed in the above example code, there is no way for us to let the code to know the ownership, and it is unsafe to use the `data` object after `worker2` is called.  The way C++ handles the situation is to use smart pointers.

# `std::unique_ptr`

(Modern) C++ provides two smart pointers: `unique_ptr` and `shared_ptr`.  We start with `unique_ptr` because it is lighter-weight.  A `unique_ptr` takes the same number of bytes of a raw pointer.  It may be a drop-in replace with a raw pointer.

`unique_ptr` should be used when there can only be one owner of the pointed object.

See the example code `01_pointer/04_unique.cpp`.

```cpp
static_assert(sizeof(Data *) == sizeof(std::unique_ptr<Data>), "unique_ptr should take only a word");

std::unique_ptr<Data> worker1()
{
    // Create a new Data object.
    std::unique_ptr<Data> data = std::make_unique<Data>();

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

void worker2(std::unique_ptr<Data> data)
{
    if (data->is_manipulated())
    {
        data.reset();
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    std::unique_ptr<Data> data = worker1();
    std::cout << "Data pointer after worker 1: " << data.get() << std::endl;

#ifdef COPYNOWORK
    worker2(data);
#else
    worker2(std::move(data));
#endif
    std::cout << "Data pointer after worker 2: " << data.get() << std::endl;

    data.reset();
    std::cout << "Data pointer after delete: " << data.get() << std::endl;
}
```

In [ ]:
!make -C 01_pointer 04_unique ; 01_pointer/04_uniquec

In [ ]:
# Compilation fails because unique_ptr isn't copyable.
!make -C 01_pointer clean
!env FLAGS=-DCOPYNOWORK make -C 01_pointer 04_unique

# `std::shared_ptr`

Unlike `unique_ptr`, `shared_ptr` allows multiple owners.  It maintains a reference counter.  When the `std::shared_ptr` object is constructed, the counter increments.  When the pointer object (note, not the pointed object) is destructed, the counter decrements.  When the counter decrements from 1, the pointed object gets destructed.

`std::shared_ptr` provides `use_count()` function for showing the reference counts.  This reference counting technique is commonplace for managing ownership, and it appears in many other languages.

See the example code in `01_pointer/05_shared`.

```cpp
static_assert(sizeof(Data *) < sizeof(std::shared_ptr<Data>), "shared_ptr uses more than a word");

std::shared_ptr<Data> worker1()
{
    // Create a new Data object.
    std::shared_ptr<Data> data = std::make_shared<Data>();

    std::cout << "worker 1 data.use_count(): " << data.use_count() << std::endl;

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

void worker2(std::shared_ptr<Data> data)
{
    std::cout << "worker 2 data.use_count(): " << data.use_count() << std::endl;

    if (data->is_manipulated())
    {
        data.reset();
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = worker1();
    std::cout << "Data pointer after worker 1: " << data.get() << std::endl;

    worker2(data);
    std::cout << "Data pointer after worker 2: " << data.get() << std::endl;

    data.reset();
    std::cout << "Data pointer after reset from outside: " << data.get() << std::endl;
    std::cout << "main data.use_count(): " << data.use_count() << std::endl;
}
```

In [ ]:
!make -C 01_pointer 05_shared ; 01_pointer/05_shared

# Raw pointers vs smart pointers

The rule of thumb is to always start with smart pointers.  When in doubt, use `unique_ptr`.  `unique_ptr` forces a developer to think clearly about whether or not multiple owners are necessary.  Only use `shared_ptr` when it is absolutely necessary.  The reference counter is much more expensive than it looks.

# Make `Data` exclusively managed by `shared_ptr`

Sometimes we know a big resource (our `Data` class) must not be constructed and destructed lightly, and it must be managed by a `shared_ptr`.  The overhead of reference counting is negligible compare to other operations or we simply have to tolerate.  In this case, we do not want anyone to directly call the `Data` constructor:

```cpp
// We want to forbid it.
Data * raw_pointer = new Data;
```

And allow only the `shared_ptr` construction:

```cpp
// We want this to work:
std::shared_ptr<Data> sptr1(new Data);
// Or this:
std::shared_ptr<Data> sptr2 = std::make_shared<Data>();
```

The problem is, if `new Data` is forbidden, `std::shared_ptr<Data>(new Data)` is forbidden too.  How can we only turn off the first but not the second?

One idea is to use private constructor:

```cpp
class Data
{
private:
    Data() {}
public:
    static std::shared_ptr<Data> make()
    {
        std::shared_ptr<Data> ret(new Data);
        return ret;
    }
};

std::shared_ptr<Data> func()
{
#ifdef THISDOESNOTWORK
    std::shared_ptr<Data> data(new Data);
#else
    std::shared_ptr<Data> data = Data::make();
#endif
    // Do work.
    return ret;
}
```

This works because the constructor is called from the `Data` class, in the line of `std::shared_ptr<Data> ret(new Data)`.

Now consider `std::shared_ptr<Data> ret = std::make_shared<Data>()`, the private constructor fails to compile, because the function template `std::make_shared` cannot access the private constructor!  We need to work it out.  You may think of `friend`.  In some cases it works, but note that `std::make_shared` is a function template, not a function.  Friendship to a function template is not straight-forward.  Moreover, `std::make_shared` does a lot of things behind the scene.  Simply makeing friend with that function template may or may not work.  It depends on how STL implement `std::make_shared`.

A more sound resolution is to use the passkey pattern:

```cpp
class Data
{
private:
    class ctor_passkey {};
public:
    static std::shared_ptr<Data> make()
    {
        std::shared_ptr<Data> ret = std::make_shared<Data>(ctor_passkey());
        return ret;
    }
    Data() = delete;
    Data(ctor_passkey const &) {}
    // TODO: Copyability and moveability should be considered, but we leave
    // them for now.
};

std::shared_ptr<Data> worker1()
{
    // Create a new Data object.
    std::shared_ptr<Data> data;

#ifdef CTORNOWORK
    data = std::shared_ptr<Data>(new Data);
#endif

#ifdef MAKENOWORK
    data = std::make_shared<Data>();
#endif

    data = Data::make();

    std::cout << "worker 1 data.use_count(): " << data.use_count() << std::endl;

    // Manipulate the Data object.
    manipulate_with_reference(*data);

    return data;
}

void worker2(std::shared_ptr<Data> data)
{
    std::cout << "worker 2 data.use_count(): " << data.use_count() << std::endl;

    if (data->is_manipulated())
    {
        data.reset();
    }
    else
    {
        manipulate_with_reference(*data);
    }
}

int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = worker1();
    std::cout << "Data pointer after worker 1: " << data.get() << std::endl;

    worker2(data);
    std::cout << "Data pointer after worker 2: " << data.get() << std::endl;

    data.reset();
    std::cout << "Data pointer after reset from outside: " << data.get() << std::endl;
    std::cout << "main data.use_count(): " << data.use_count() << std::endl;
}

static_assert(sizeof(Data *) * 2 == sizeof(std::shared_ptr<Data>), "shared_ptr should use two word");
```

In [ ]:
!make -C 02_shared 01_fully ; 02_shared/01_fully

In [ ]:
# The deleted constructor fails with shared_ptr constructor, as expected
!make -C 02_shared clean
!env FLAGS=-DCTORNOWORK make -C 02_shared 01_fully

In [ ]:
# The deleted constructor fails with make_shared, as expected
!make -C 02_shared clean
!env FLAGS=-DMAKENOWORK make -C 02_shared 01_fully

# Get `shared_ptr` from `this`

Occasionally we get the `Data` object without the `shared_ptr` that manages it, but want to give the ownership to the caller:

```cpp
class Data
{
public:
    Data * get_raw_ptr()
    {
        // Returning raw pointer discards the ownership management.
        return this;
    }

    std::shared_ptr<Data> get_shared_ptr()
    {
        // XXX: Recreate a shared_ptr will duplicate the reference counter, and
        // later results into double free.
        return std::shared_ptr<Data>(this);
    }
};
```

The above code (`get_shared_ptr()`) naively creates an alternate `shared_ptr` object, and will results in double free in the caller:

```cpp
int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = Data::make();
    std::cout << "data.use_count(): " << data.use_count() << std::endl;

#ifdef DUPSHARED
    std::shared_ptr<Data> holder2 = data->get_shared_ptr();
#endif

    data.reset();
    std::cout << "data.use_count() after data.reset(): " << data.use_count() << std::endl;

#ifdef DUPSHARED
    std::cout << "holder2.use_count(): " << holder2.use_count() << std::endl;
    holder2.reset();
    // This line never gets reached since the the above line causes double free
    // and crash.
    std::cout << "holder2.use_count() after holder2.reset(): " << holder2.use_count() << std::endl;
#endif
}
```

In [ ]:
!make -C 02_shared 02_duplicate
# This will crash with double free.
!02_shared/02_duplicate

## Never recreate a `shared_ptr` from raw pointer

The right way to get a `std::shared_ptr` from a `shared_ptr`-managed object is to use `std::enable_shared_from_this`:

```cpp
class Data
  : public std::enable_shared_from_this<Data>
{
public:
    std::shared_ptr<Data> get_shared_ptr()
    {
        // This is the right way to get the shared pointer from within the
        // object.
        return shared_from_this();
    }
};

int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = Data::make();
    std::cout << "data.use_count(): " << data.use_count() << std::endl;

    std::shared_ptr<Data> holder2 = data->get_shared_ptr();
    std::cout << "data.use_count() after holder2: " << data.use_count() << std::endl;

    data.reset();
    std::cout << "data.use_count() after data.reset(): " << data.use_count() << std::endl;

    std::cout << "holder2.use_count() before holder2.reset(): " << holder2.use_count() << std::endl;
    holder2.reset();
    std::cout << "holder2.use_count() after holder2.reset(): " << holder2.use_count() << std::endl;
}
```

In [ ]:
!make -C 02_shared 03_fromthis ; 02_shared/03_fromthis

# Cyclic reference!

When two object use a pair of `shared_ptr` to point to each other, the cyclic reference will create a memory leak:

```cpp
class Data
  : public std::enable_shared_from_this<Data>
{
public:
    std::shared_ptr<Child>   child() const { return m_child; }
    std::shared_ptr<Child> & child()       { return m_child; }
private:
    std::shared_ptr<Child> m_child;
};

class Child
  : public std::enable_shared_from_this<Child>
{
private:
    class ctor_passkey {};
public:
    Child() = delete;
    Child(std::shared_ptr<Data> const & data, ctor_passkey const &) : m_data(data) {}
    static std::shared_ptr<Child> make(std::shared_ptr<Data> const & data)
    {
        std::shared_ptr<Child> ret = std::make_shared<Child>(data, ctor_passkey());
        data->child() = ret;
        return ret;
    }
private:
    std::shared_ptr<Data> m_data;
};

int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = Data::make();
    std::shared_ptr<Child> child = Child::make(data);
    std::cout << "data.use_count(): " << data.use_count() << std::endl;
    std::cout << "child.use_count(): " << child.use_count() << std::endl;

    std::weak_ptr<Data> wdata(data);
    std::weak_ptr<Child> wchild(child);

    data.reset();
    std::cout << "wdata.use_count() after data.reset(): " << wdata.use_count() << std::endl;
    std::cout << "wchild.use_count() after data.reset(): " << wchild.use_count() << std::endl;

    child.reset();
    std::cout << "wdata.use_count() after child.reset(): " << wdata.use_count() << std::endl;
    std::cout << "wchild.use_count() after child.reset(): " << wchild.use_count() << std::endl;
    // Oops, the reference count doesn't reduce to 0!
}
```

In [ ]:
!make -C 02_shared 04_cyclic ; 02_shared/04_cyclic

 ## Use `std::weak_ptr` to break cyclic reference
 
In the above demonstration we use `weak_ptr` to get the reference count without increasing it.  `weak_ptr` can also be used to break the cyclic reference.  In the following example, the `Child` object replaces `shared_ptr` with `weak_ptr` to point to `Data`:

```cpp
class Child
  : public std::enable_shared_from_this<Child>
{
private:
    class ctor_passkey {};
public:
    Child() = delete;
    Child(std::shared_ptr<Data> const & data, ctor_passkey const &) : m_data(data) {}
    static std::shared_ptr<Child> make(std::shared_ptr<Data> const & data)
    {
        std::shared_ptr<Child> ret = std::make_shared<Child>(data, ctor_passkey());
        data->child() = ret;
        return ret;
    }
private:
    // Replace shared_ptr with weak_ptr to Data.
    std::weak_ptr<Data> m_data;
};

int main(int argc, char ** argv)
{
    std::shared_ptr<Data> data = Data::make();
    std::shared_ptr<Child> child = Child::make(data);
    std::cout << "data.use_count(): " << data.use_count() << std::endl;
    std::cout << "child.use_count(): " << child.use_count() << std::endl;

    std::weak_ptr<Data> wdata(data);
    std::weak_ptr<Child> wchild(child);

    child.reset();
    std::cout << "wdata.use_count() after child.reset(): " << wdata.use_count() << std::endl;
    std::cout << "wchild.use_count() after child.reset(): " << wchild.use_count() << std::endl;

    data.reset();
    std::cout << "wdata.use_count() after data.reset(): " << wdata.use_count() << std::endl;
    std::cout << "wchild.use_count() after data.reset(): " << wchild.use_count() << std::endl;
}
```

In [ ]:
!make -C 02_shared 05_weak ; 02_shared/05_weak

## But actually we should avoid `weak_ptr`

Using `weak_ptr` to break cyclic reference should only be considered as a workaround, rather than a full resolution.  We sometimes need it since the refenrece cycle may not be as obvious as it is in our example.  For example, there may be 3 or 4 levels of references in the cycle.  `weak_ptr` has a similar interface to `shared_ptr`.  When we are troubleshooting resource-leaking issues, replacing `shared_ptr` with `weak_ptr` can work as a quick-n-dirty hotfix.

The right treatment is to sort out the ownership.  It's not easy when the system is complex.  The rule of thumb is that, as we mentioned earlier, you should avoid using `shared_ptr` unless you really need it.  And most of the time the need appears in a higher-level and heavy-weight container, rather than the lower-level small objects.  For small objects, we should try to limit the lifecycle and use raw pointers or stack.

# Exercises

1. Write code so that when `std::unique_ptr` is destructed, the object it points to doesn't destruct.
2. Create vectors of 1,000,000 elements of (i) raw pointers, (ii) `unique_ptr`, and (iii) `shared_ptr`, respectively, and measure difference of the performance.
3. Compare the runtime performance between `shared_ptr(new Type)` and `make_shared<Type>`.  Explain why there is a difference of performance.